<img src="header.png" align="left"/>

# Anwendungsbeispiel Import of excel data and timeseries prediction

Das Ziel dieses Beispieles ist es die Arbeit mit Zeitreihen aus Excel Files zu zeigen und darauf ein Vorhersagemodell für Zeitreihen zu entwickeln. Der Datensatz beschreibt die Entwicklung von Flugpassagierzahlen einer Airline in der fernen Vergangenheit. Die Daten sind aber typisch für Daten wie sie KMUs (z.B. Verkaufszahlen) zu finden sind. 


Der Code für das Beispiel wurde aus [1] adaptiert. Weitergehende Informationen sind zum Beispiel in [2] zu finden. 

- [1] [https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/](https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/)
- [2] [https://towardsdatascience.com/predict-electricity-consumption-using-time-series-analysis-4650284e40aa](https://towardsdatascience.com/predict-electricity-consumption-using-time-series-analysis-4650284e40aa)


Zitierung Datensatz:
```
Box, G. E. P., Jenkins, G. M. and Reinsel, G. C. (1976) Time Series Analysis, Forecasting and Control. Third Edition. Holden-Day. Series G.
```


# Import der Module

In [ ]:
#
# Import der Module
#
import os
import openpyxl

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

plt.rcParams["figure.figsize"] = (16,9)


In [ ]:
#
# Abdrehen von Fehlermeldungen
#
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=Warning)

In [ ]:
#
# Für GPU Support
#
import tensorflow as tf
print ( tf.__version__ ) 

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR )
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

# Konstanten

In [ ]:
#
# Konstanten für Dateien
#
excelData = 'data/airline_passengers.xlsx'

In [ ]:
data = pd.ExcelFile(excelData)
print(data.sheet_names)

In [ ]:
#
# Auslesen eines Tabellenblattes
#
df = data.parse('Tabellenblatt1')
df.info
df.head(10)

In [ ]:
df.tail()

# Ausschneiden von Daten in pandas dataframes

Ein sehr gutes Tutorial dazu kann hier gefunden werden: [https://www.shanelynn.ie/select-pandas-dataframe-rows-and-columns-using-iloc-loc-and-ix/](https://www.shanelynn.ie/select-pandas-dataframe-rows-and-columns-using-iloc-loc-and-ix/)

<img src="info.png" align="left"/> 

In [ ]:
#
# Ausschneiden der benötigten Daten
#
df_cut = df.iloc[3:146,1:3]

In [ ]:
df_cut.head()

In [ ]:
df_cut.columns = ['month','passengers']

In [ ]:
df_cut.head()

In [ ]:
df_cut['passengers'].plot()

# Konventionelle Arbeit mit solchen Zeitreihen

Es gibt eine Reihe von älteren Methoden um mit solchen Zeitreihen umzugehen und auch gute Ergebnisse bei der Vorhersage zu erreichen. Beispiele dazu sind hier zu finden:

- https://machinelearningmastery.com/time-series-trends-in-python/
- https://towardsdatascience.com/predict-electricity-consumption-using-time-series-analysis-4650284e40aa


Die wesentliche Leistung dieser Verfahren ist die Berechnung und Verwendung von Parameters wie **Trend** und **Saisonalität**. Wir hoffen, dass unsere Modelle damit umgehen können, ohne dass wir uns explizit darum kümmern müssen.


<img src="info.png" align="left"/> 

In [ ]:
x_data = df_cut['passengers'].values
x_data = x_data.astype('float32')
x_data = np.reshape(x_data,(-1,1))
print(x_data)

In [ ]:
#
# Normalize the dataset
#
scaler = MinMaxScaler(feature_range=(0.0, 1.0))
scaler.fit(x_data)

In [ ]:
print(scaler.data_max_)

In [ ]:
x_data = scaler.transform(x_data)
print(x_data)

In [ ]:
# split into train and test sets
train_size = int(len(x_data) * 0.80)
test_size = len(x_data) - train_size
train, test = x_data[0:train_size,:], x_data[train_size:len(x_data),:]
print(len(train), len(test))

# Erzeugen eines Trainingsdatensatzes mit Hilfe von sliding windows

Sliding windows werden aus einer Zeitreihe erzeugt, indem ein Fenster über die gesamte Zeitreihe gezogen wird und dabei jeweils die Daten im Fenster kopiert werden. Mehr Details dazu hier [https://towardsdatascience.com/ml-approaches-for-time-series-4d44722e48fe](https://towardsdatascience.com/ml-approaches-for-time-series-4d44722e48fe).

<img src="info.png" align="left"/> 

In [ ]:
#
# Transformation einer Zeitreihe in sliding windows mit einem label (y)
#
def createSlidingWindowsWithLabel(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[ i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

In [ ]:
#
# Anwenden der sliding window Funktion
#
look_back = 1
trainX, trainY = createSlidingWindowsWithLabel(train, look_back)
testX, testY = createSlidingWindowsWithLabel(test, look_back)

In [ ]:
# 
# reshape input to be [samples, time steps, features]
#
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [ ]:
#
# create and fit the LSTM network
#
def createLSTMModel():
    model = Sequential()
    model.add(LSTM(4, input_shape=(1, look_back)))
    model.add(Dense(1,activation='linear'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [ ]:
lstm_model = createLSTMModel()

In [ ]:
lstm_model.fit(trainX, trainY, epochs=40, batch_size=1, verbose=1)

In [ ]:
#
# Schätzung der Werte für train und test Daten
#
trainPredict = lstm_model.predict(trainX)
testPredict = lstm_model.predict(testX)

In [ ]:
#
# Rücktransformation der Schätzungen (scaler)
#
trainPredict = scaler.inverse_transform(trainPredict)
trainYi = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testYi = scaler.inverse_transform([testY])

In [ ]:
#
# Anzeige der geschätzten neuen Werte
#
plt.plot(testYi[0,0:])
plt.plot(testPredict[1:,0])
plt.show()

In [ ]:
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainYi[0,0:-1], trainPredict[1:,0]))
print('train loss: %.3f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testYi[0,0:-1], testPredict[1:,0]))
print('test loss: %.3f RMSE' % (testScore))

In [ ]:
# shift train predictions for plotting
trainPredictPlot = np.empty_like(x_data)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[:len(trainPredict)+0, :] = trainPredict

# shift test predictions for plotting
testPredictPlot = np.empty_like(x_data)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2):len(x_data)-2, :] = testPredict

In [ ]:
# plot baseline and predictions
plt.plot(scaler.inverse_transform(x_data), color='grey')
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

# Verbesserung durch Verwendung von Zusatzinformation in den Daten

**Idee**: In dem Datensatz stecken mehr Informationen als wir bisher verwendet haben. Zum Beispiel das Monat eines Jahres sowie das Jahr. Wir können den Datensatz um diese Information anreichern um dem Modell mehr Informationen zu geben. Diese zusätzlich Information können wir aus dem Datumsfeature in den Originaldaten ableiten.

In [ ]:
df_cut.head()

In [ ]:
df_cut['date'] = pd.to_datetime(df_cut['month'], infer_datetime_format=True )

In [ ]:
df_cut.head()

In [ ]:
df_cut['year'] = pd.DatetimeIndex(df_cut['date']).year
df_cut['month'] = pd.DatetimeIndex(df_cut['date']).month

In [ ]:
df_cut.head()

In [ ]:
x_data_raw = df_cut.loc[:, ['month', 'passengers','year']].values
print(x_data_raw)

In [ ]:
x_data = x_data_raw.astype('float32')

In [ ]:
x_data = np.reshape(x_data,(-1,3))

In [ ]:
print(x_data)

In [ ]:
#
# Normalize the dataset
#
scaler = MinMaxScaler(feature_range=(0.0, 1.0))
scaler.fit(x_data)

In [ ]:
x_data_scaled = scaler.transform(x_data)
print(x_data_scaled)

In [ ]:
plt.plot(x_data_scaled)

In [ ]:
#
# Transformation einer multivariaten Zeitreihe in sliding windows mit einem label (y)
#
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('x%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('y%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('y%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
            
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg
 


In [ ]:
#
# Erzeugen der Windows für Vorhersage
#
windowSize = 12
x_data_windowed = series_to_supervised(x_data_scaled, windowSize, 1)
print(x_data_windowed.head())

In [ ]:
#
# Löschen der Labels, die nicht geschätzt werden sollen (month, year)
#

x_data_windowed.drop(x_data_windowed.columns[[windowSize*3,(windowSize*3)+2]], axis=1, inplace=True)
print(x_data_windowed.head())

In [ ]:
#x_data_windowed.plot()

In [ ]:
# split into train and test sets
x_data = x_data_windowed.values
train_size = int(len(x_data) * 0.70)
test_size = len(x_data) - train_size
train, test = x_data[0:train_size,:], x_data[train_size:len(x_data),:]
print(train.shape, test.shape)

In [ ]:
#
# Trennen in features und labels
#
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]

In [ ]:
#
# Form anpassen für Modell
#
train_X_r = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X_r = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X_r.shape, train_y.shape, test_X_r.shape, test_y.shape)

In [ ]:
#
# create and fit the LSTM network
#
def createMVLSTMModel():
    model = Sequential()
    model.add(LSTM(100, dropout=0.1, input_shape=(train_X_r.shape[1], train_X_r.shape[2])))
    model.add(Dense(1,activation='linear'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [ ]:
model = createMVLSTMModel()
history = model.fit(train_X_r, train_y, epochs=300, batch_size=24, validation_data=(test_X_r, test_y), verbose=1, shuffle=False)    

In [ ]:
# plot history
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
#
# Berechung der Vorhersagen
#
yhat = model.predict(test_X_r,batch_size=1)
trainyhat = model.predict(train_X_r,batch_size=1)

In [ ]:
def invertForecastScaling(x,y):
    # invert scaling for forecast
    inv_yhat = np.concatenate( ( x[:,0:1], y[:,:], x[:, 0:1] ), axis=1 )
    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:,1]
    return inv_yhat

In [ ]:
inv_yhat = invertForecastScaling(test_X,yhat)
print(inv_yhat)

In [ ]:
inv_y = invertForecastScaling(test_X,test_y.reshape(-1,1))
print(inv_y)

In [ ]:
plt.plot(inv_yhat,color='b')
plt.plot(inv_y,color='g')
plt.show()

In [ ]:
# calculate RMSE
testScore = np.sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test loss: %.3f RMSE' % testScore)

In [ ]:
inv_y_train = invertForecastScaling(train_X,trainyhat.reshape(-1,1))

In [ ]:
# shift train predictions for plotting
trainPredictPlot = np.empty_like(x_data_raw[:,1])
trainPredictPlot[:] = np.nan
trainPredictPlot[windowSize:len(inv_y_train)+windowSize] = inv_y_train

In [ ]:
# shift test predictions for plotting
testPredictPlot = np.empty_like(x_data_raw[:,1])
testPredictPlot[:] = np.nan
testPredictPlot[len(x_data_raw)-len(inv_yhat):len(x_data_raw)] = inv_yhat

In [ ]:
# plot baseline and predictions
plt.plot(x_data_raw[:,1])
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

## Welche Verbesserungsschritte wären sonst noch möglich?